In [1]:
!pip install duckdb
import duckdb
from datetime import datetime
con = duckdb.connect('/tmp/DuckDB Database')
c = con.cursor()

     |████████████████████████████████| 1.3MB 2.8MB/s 
  Created wheel for duckdb: filename=duckdb-0.1.1-cp36-cp36m-linux_x86_64.whl size=1947901 sha256=58d064d3189ed97fe130c5c40d2c2d67d96ce0a81815ea523f94dd2f14f53514
  Stored in directory: /root/.cache/pip/wheels/31/2e/81/8061e62cf80a0ea73a4657d5807c46a93105440af5921e828c
Successfully built duckdb


In [2]:
!wget -O Robust04.rar https://dl.dropboxusercontent.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar?dl=0
!unrar e -o+ Robust04.rar

--2019-11-18 10:11:43--  https://dl.dropboxusercontent.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar?dl=0
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.82.6, 2620:100:6032:6::a27d:5206
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.82.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460221412 (439M) [application/rar]
Saving to: ‘Robust04.rar’

Robust04.rar        100%[===================>] 438.90M  41.7MB/s    in 12s     

2019-11-18 10:11:56 (36.2 MB/s) - ‘Robust04.rar’ saved [460221412/460221412]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Robust04.rar

Extracting  terms.csv                                                      0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28%

In [3]:
c.execute("CREATE TABLE dict(termid INTEGER "
                           ",term   VARCHAR "
                           ",df     INTEGER "
                           ",cf     INTEGER)")
c.execute("CREATE TABLE terms(termid INTEGER "
                            ",docid  INTEGER "
                            ",pos    INTEGER "
                            ",tf     INTEGER)")
c.execute("CREATE TABLE docs(name  VARCHAR "
                           ",docid INTEGER "
                           ",len   INTEGER "
                           ",temp  INTEGER)")

In [4]:
c.execute("COPY dict "
          "FROM 'dict.csv' "
          "WITH DELIMITER '|'")
c.execute("COPY terms "
          "FROM 'terms.csv' "
          "WITH DELIMITER '|'")
c.execute("COPY docs "
          "FROM 'docs.csv' "
          "WITH DELIMITER '|'")

In [0]:
def make_query(c, *args: str):
    """Function to make query table in database.
    
    Args:
        c: cursor of DuckDB
        args: concatenation of strings to be made into a query
        
    Returns [str]:
        Number of terms in the query as string.
    """
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    c.execute("DROP TABLE IF EXISTS query")
    c.execute("CREATE TABLE query(term VARCHAR)")
    c.execute("INSERT INTO query VALUES " + query)     
        
    return str(len(args))

In [6]:
len_query = make_query(c, 'wizard', 'hat')

cur_time = datetime.now()

qterms = c.execute("WITH qtermids "
                        "AS (SELECT dict.termid "
                                  ",dict.df "
                                  ",dict.cf "
                            "FROM dict "
                            "JOIN query "
                            "ON dict.term = query.term "
                            ") "
                   "SELECT terms.termid "
                         ",terms.docid "
                         ",terms.pos "
                         ",terms.tf "
                         ",qtermids.df "
                         ",qtermids.cf "
                   "FROM terms "
                   "JOIN qtermids "
                   "ON terms.termid = qtermids.termid")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

qterms.fetchdf()

Query time: 0:00:01.443901


,termid,docid,pos,tf,df,cf
0,434301,44,589,1,1718,2250
1,434301,152,497,1,1718,2250
2,434301,281,92,1,1718,2250
3,434301,339,1024,1,1718,2250
4,434301,396,696,1,1718,2250
...,...,...,...,...,...,...
2590,694497,323249,53,2,269,345
2591,694497,323249,186,2,269,345
2592,694497,324788,1341,1,269,345
2593,694497,326675,155,1,269,345


In [10]:
len_query = make_query(c, 'new', 'york')

C = c.execute("SELECT SUM(dict.cf) "
              "FROM dict")
C = str(float(C.fetchdf().iloc[0, 0]))
mu = '0.8' # very uncertain of this hyperparameter

N = c.execute("SELECT COUNT(*) "
              "FROM docs")
N = str(N.fetchdf().iloc[0, 0])
avgdl = c.execute("SELECT AVG(docs.len) "
                  "FROM docs")
avgdl = str(avgdl.fetchdf().iloc[0, 0])
k1 = '1.2'
b = '0.75'

k = '20'

maxspan = '5'

cur_time = datetime.now()

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                             ",dict.cf "
                       "FROM dict "
                       "JOIN query "
                       "ON dict.term = query.term "
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",terms.tf "
                             ",qtermids.df "
                             ",qtermids.cf "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              ", qtermstf "
                   "AS (SELECT qterms.docid "
                             ",qterms.tf "
                             ",qterms.df "
                             ",qterms.cf "
                             ",( ROW_NUMBER() "
                                "OVER(PARTITION BY qterms.termid, qterms.docid "
                                     "ORDER BY qterms.pos"
                                     ")"
                               ") AS row "
                       "FROM qterms "
                      #  "JOIN condocs "
                      #  "ON qterms.docid = condocs.docid "  
                       ") "
              ", okapisubscores "
               "AS (SELECT qtermstf.docid "
                         ",( LOG((" + N + "-df+0.5)/(df+0.5))*tf*(" + k1 + "+1)"
                             "/"
                            "(tf+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                           ") AS subscore "
                   "FROM qtermstf "
                   "JOIN docs "
                   "ON qtermstf.docid = docs.docid "
                   "WHERE qtermstf.row = 1"
                   ") "                         
              ", kldsubscores "
                   "AS (SELECT qtermstf.docid "
                            ",( LOG(" + mu + "+tf*" + C + "/cf)"              
                                "+" 
                               "LOG(1/(" + mu + "+len))"
                              ") AS subscore "
                       "FROM qtermstf "
                       "JOIN docs "
                       "ON qtermstf.docid = docs.docid "
                       "WHERE qtermstf.row = 1"
                       ") "
              ", topkdocs "
                   "AS (SELECT subscores.docid "
                             ",SUM(subscores.subscore) AS score "
                       "FROM kldsubscores AS subscores "
                      #  "FROM okapisubscores AS subscores "
                       "GROUP BY subscores.docid "
                       "ORDER BY SUM(subscores.subscore) DESC "
                       "LIMIT " + k +
                       ") "
              ", pairs "
                   "AS (SELECT qterms1.termid AS termid1 "
                             ",qterms2.termid AS termid2 "
                             ",qterms1.docid "
                             ",1.0/(qterms1.pos-qterms2.pos) AS tpi "
                             ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                     "ELSE qterms2.df "
                                "END"
                               ") AS maxdf "
                       "FROM qterms AS qterms1 "
                      #  "JOIN condocs "
                      #  "ON qterms1.docid = condocs.docid "
                       "JOIN qterms AS qterms2 "
                       "ON qterms1.docid = qterms2.docid AND "
                          "NOT qterms1.termid = qterms2.termid AND "
                          "qterms1.pos - qterms2.pos BETWEEN 1 AND " + maxspan +
                       "JOIN topkdocs "
                       "ON qterms1.docid = topkdocs.docid"
                       ") "
              ",tpisums "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",SUM(pairs.tpi) AS tpisum "
                       "FROM pairs "              
                       "GROUP BY pairs.termid1 "
                               ",pairs.termid2 "
                               ",pairs.docid"
                       ") "
              ", tpsubscores "
                   "AS (SELECT pairs.docid "
                             ",( LOG((" + N + "-maxdf+0.5)/(maxdf+0.5))*tpisum*(" + k1 + "+1)"
                                 "/"
                                "(tpisum+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                               ") AS subscore "
                       "FROM pairs "
                       "JOIN tpisums "
                       "ON pairs.termid1 = tpisums.termid1 AND "
                          "pairs.termid2 = tpisums.termid2 AND "
                          "pairs.docid = tpisums.docid "
                       "JOIN docs "
                       "ON pairs.docid = docs.docid"
                       ") "
              ", tpscores "
                   "AS (SELECT tpsubscores.docid "
                             ",SUM(tpsubscores.subscore) AS tpscore "
                       "FROM tpsubscores "
                       "GROUP BY tpsubscores.docid"
                       ") "
              ", scores "
                   "AS (SELECT topkdocs.docid "
                             ",( topkdocs.score"
                                 "+"
                                "COALESCE(tpscores.tpscore, 0)"
                               ") AS score "
                       "FROM topkdocs "
                       "LEFT JOIN tpscores "
                       "ON topkdocs.docid = tpscores.docid"
                       ") "
              "SELECT scores.docid "
                    ",scores.score "
              "FROM scores "
              # "JOIN topkdocs "
              # "ON scores.docid = topkdocs.docid "
              # "WHERE scores.docid BETWEEN 50961 AND 84012 "
              "ORDER BY scores.score DESC")               

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

t.fetchdf()

Query time: 0:00:09.467267


,docid,score
0,50961,59.406660
1,100831,18.508918
2,330675,13.951959
3,254096,11.489001
4,323541,10.929049
5,187978,8.859076
6,60420,7.729182
7,256566,6.604098
8,134955,5.923230
9,111825,5.485813
